In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os

# Load the PLY point cloud data using Open3D
ply_file_path = r"C:\Users\wenru\Desktop\POLYCAM\Birnam Oak-pc-poly\point_cloud.ply"
point_cloud = o3d.io.read_point_cloud(ply_file_path)

# Convert point cloud data to a NumPy array for easier manipulation
points = np.asarray(point_cloud.points)

# Normalize the Z-axis by setting the lowest point to Z=0
z_min = points[:, 2].min()
points[:, 2] -= z_min

# Get the range of X and Y coordinates to define the bounding box
x_min, x_max = points[:, 0].min(), points[:, 0].max()
y_min, y_max = points[:, 1].min(), points[:, 1].max()

# Define the thickness of each slice in meters (10 cm per slice)
slice_thickness = 0.1

# Get the maximum Z value in the normalized point cloud
z_max = points[:, 2].max()

# Calculate the total number of slices based on the Z range and slice thickness
num_slices = int(z_max / slice_thickness) + 1

# Initialize a list to store slices and metadata
slices = []

# Loop through each slice and extract the corresponding points
for i in range(num_slices):
    z_start = i * slice_thickness
    z_end = z_start + slice_thickness
    
    # Select points that fall within the current slice range
    slice_points = points[(points[:, 2] >= z_start) & (points[:, 2] < z_end)]
    
    # If the slice contains points, store the slice data
    if slice_points.size > 0:
        slices.append({
            'slice': i,
            'z_start': z_start,
            'z_end': z_end,
            'points': slice_points
        })

# Create a directory based on the input PLY file name to store slice visualizations and metadata
input_filename = os.path.splitext(os.path.basename(ply_file_path))[0]
output_dir = f"{input_filename}_info"
os.makedirs(output_dir, exist_ok=True)

# Initialize a list to store metadata for each slice
metadata = []

# Loop through each slice for visualization and metadata extraction
for i in range(num_slices):
    z_start = i * slice_thickness
    z_end = z_start + slice_thickness
    
    # Select points that fall within the current slice range
    slice_points = points[(points[:, 2] >= z_start) & (points[:, 2] < z_end)]
    
    # If the slice contains points, create a visualization and save metadata
    if slice_points.size > 0:
        # Create a figure for visualization with an equal aspect ratio
        plt.figure(figsize=(8, 8))
        plt.scatter(slice_points[:, 0], slice_points[:, 1], s=1, color='black')
        plt.axis('off')  # Hide the axis labels for a clean image
        plt.xlim(x_min, x_max)  # Maintain a consistent x-axis range across all slices
        plt.ylim(y_min, y_max)  # Maintain a consistent y-axis range across all slices
        plt.gca().set_aspect('equal', adjustable='box')

        # Save the visualization as a PNG file with the slice number and height range in the filename
        slice_filename = f'slice_{i}_({z_start:.2f} to {z_end:.2f}m).png'
        plt.savefig(os.path.join(output_dir, slice_filename), bbox_inches='tight', pad_inches=0)
        plt.close()

        # Store metadata such as the bounding box and slice number
        metadata.append({
            'slice': i,
            'bbox': [x_min, y_min, x_max, y_max]  # Save the original bounding box
        })

# Save the entire point cloud data and slice metadata to .npy files for future use
np.save(os.path.join(output_dir, f"{output_dir}_points.npy"), points)
np.save(os.path.join(output_dir, f"{output_dir}_metadata.npy"), metadata)

# Output completion message
print("Step 1 completed. All slices saved.")



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Step 1 completed. All slices saved.


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.tri as mtri
from shapely.geometry import Polygon

# Global variables for polygon drawing
drawing = False
polygon_points = []
all_polygons = []

# Define output directory for processed images
new_output_dir = "processed_point_cloud"
os.makedirs(new_output_dir, exist_ok=True)

# Load metadata from previous slicing step
metadata_filename = os.path.join(output_dir, f"{output_dir}_metadata.npy")
metadata = np.load(metadata_filename, allow_pickle=True)

def convert_to_physical_coords(x, y, img_width, img_height, x_min, x_max, y_min, y_max):
    """
    Convert image coordinates to physical coordinates based on the bounding box and image dimensions.
    """
    physical_x = x_min + (x / img_width) * (x_max - x_min)
    physical_y = y_max - (y / img_height) * (y_max - y_min)
    return physical_x, physical_y

def convert_to_image_coords(px, py, img_width, img_height, x_min, x_max, y_min, y_max):
    """
    Convert physical coordinates to image coordinates based on the bounding box and image dimensions.
    """
    image_x = int((px - x_min) / (x_max - x_min) * img_width)
    image_y = int((y_max - py) / (y_max - y_min) * img_height)
    return image_x, image_y

# Mouse callback function for drawing polygons interactively
def draw_polygon(event, x, y, flags, param):
    global drawing, polygon_points, img_with_axis, x_min, y_min, x_max, y_max, img_width, img_height

    if event == cv2.EVENT_LBUTTONDOWN:
        physical_x, physical_y = convert_to_physical_coords(x, y, img_width, img_height, x_min, x_max, y_min, y_max)
        polygon_points.append((physical_x, physical_y))
        drawing = True

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp_img = img_with_axis.copy()
        if len(polygon_points) > 1:
            cv2.polylines(temp_img, [np.array([convert_to_image_coords(px, py, img_width, img_height, x_min, x_max, y_min, y_max) for (px, py) in polygon_points])], isClosed=False, color=(0, 255, 0), thickness=2)
        cv2.line(temp_img, convert_to_image_coords(polygon_points[-1][0], polygon_points[-1][1], img_width, img_height, x_min, x_max, y_min, y_max), (x, y), (0, 255, 0), 2)
        cv2.imshow('image', temp_img)

    elif event == cv2.EVENT_LBUTTONUP:
        physical_x, physical_y = convert_to_physical_coords(x, y, img_width, img_height, x_min, x_max, y_min, y_max)
        polygon_points.append((physical_x, physical_y))
        drawing = False
        temp_img = img_with_axis.copy()
        if len(polygon_points) > 1:
            cv2.polylines(temp_img, [np.array([convert_to_image_coords(px, py, img_width, img_height, x_min, x_max, y_min, y_max) for (px, py) in polygon_points])], isClosed=False, color=(0, 255, 0), thickness=2)
        cv2.imshow('image', temp_img)

def process_slice(slice_num, slice_thickness):
    """
    Process a specific slice by allowing the user to draw a polygon on the slice image.
    Calculate the area of the drawn polygon in physical units and save the result.
    """
    global x_min, y_min, x_max, y_max, img_width, img_height

    # Generate the image file name for the current slice
    image_path = os.path.join(output_dir, f"slice_{slice_num}_({slice_num*slice_thickness:.2f} to {(slice_num+1)*slice_thickness:.2f}m).png")
    if not os.path.exists(image_path):
        print(f"Slice image {slice_num} not found.")
        return 0

    # Get the metadata for the current slice
    current_metadata = next(item for item in metadata if item['slice'] == slice_num)
    x_min, y_min, x_max, y_max = current_metadata['bbox']  # Get the bounding box

    # Load the image and convert to BGR for drawing colored lines
    global img_with_axis, polygon_points
    img_with_axis = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_with_axis = cv2.cvtColor(img_with_axis, cv2.COLOR_GRAY2BGR)

    # Get image dimensions
    img_height, img_width = img_with_axis.shape[:2]

    # Add grid and axes to the image for better reference
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(img_with_axis, extent=[x_min, x_max, y_min, y_max], origin='upper')
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True)
    plt.savefig(os.path.join(new_output_dir, f'slice_{slice_num}_with_axis.png'))
    plt.close()

    # Reload the image with the axes and grid
    img_with_axis = cv2.imread(os.path.join(new_output_dir, f'slice_{slice_num}_with_axis.png'))

    polygon_points = []  # Reset the list of points

    # Create an image window and set the mouse callback function
    cv2.namedWindow('image')
    cv2.setMouseCallback('image', draw_polygon)
    cv2.imshow('image', img_with_axis)

    # Wait for the user to complete the polygon drawing
    while True:
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Exit when ESC is pressed
            break
        elif key == ord('c'):  # Finish the polygon when 'c' is pressed
            if len(polygon_points) > 2:
                break

    cv2.destroyAllWindows()

    # If the polygon has more than 2 points, calculate the area
    if len(polygon_points) > 2:
        polygon = Polygon(polygon_points)
        area = polygon.area  # Calculate the polygon's area
        print(f'Polygon area in physical units for slice {slice_num}: {area} square meters')

        # Save the polygon and convert it to 3D by adding Z coordinates
        polygon_with_z = [(x, y, slice_num * slice_thickness) for (x, y) in polygon_points]
        all_polygons.append(polygon_with_z)

        # Save the processed image with the polygon overlay
        img_with_axis = cv2.imread(os.path.join(new_output_dir, f'slice_{slice_num}_with_axis.png'))
        for x, y in polygon_points:
            cv2.circle(img_with_axis, convert_to_image_coords(x, y, img_width, img_height, x_min, x_max, y_min, y_max), 5, (0, 255, 0), -1)
        if len(polygon_points) > 1:
            cv2.polylines(img_with_axis, [np.array([convert_to_image_coords(px, py, img_width, img_height, x_min, x_max, y_min, y_max) for (px, py) in polygon_points])], isClosed=True, color=(0, 255, 0), thickness=2)

        # Save the image with the drawn polygon
        result_image_path = os.path.join(new_output_dir, f'slice_{slice_num}_processed.png')
        cv2.imwrite(result_image_path, img_with_axis)
        print(f"Processed image saved at: {result_image_path}")

        return area
    else:
        print(f"Not enough points to form a polygon for slice {slice_num}.")
        return 0

def save_all_polygons_to_ply(polygons, ply_filename):
    """
    Save all drawn polygons as a triangulated mesh into a PLY file.
    """
    all_vertices = []
    all_faces = []
    vertex_offset = 0

    for polygon in polygons:
        x_coords = [p[0] for p in polygon]
        y_coords = [p[1] for p in polygon]
        z_coords = [p[2] for p in polygon]

        # Perform Delaunay triangulation on the polygon's vertices
        triangulation = mtri.Triangulation(x_coords, y_coords)
        all_vertices.extend(polygon)

        # Create faces for the triangulated mesh
        for triangle in triangulation.triangles:
            all_faces.append([triangle[0] + vertex_offset, 
                              triangle[1] + vertex_offset, 
                              triangle[2] + vertex_offset])

        vertex_offset += len(polygon)

    # Write the vertices and faces to a PLY file
    with open(ply_filename, 'w') as ply_file:
        ply_file.write("ply\n")
        ply_file.write("format ascii 1.0\n")
        ply_file.write(f"element vertex {len(all_vertices)}\n")
        ply_file.write("property float x\n")
        ply_file.write("property float y\n")
        ply_file.write("property float z\n")
        ply_file.write(f"element face {len(all_faces)}\n")
        ply_file.write("property list uchar int vertex_indices\n")
        ply_file.write("end_header\n")

        # Write vertices
        for vertex in all_vertices:
            ply_file.write(f"{vertex[0]} {vertex[1]} {vertex[2]}\n")

        # Write faces
        for face in all_faces:
            ply_file.write(f"3 {face[0]} {face[1]} {face[2]}\n")

    print(f"All polygons saved as a single PLY file to {ply_filename}")

def save_areas_and_total_volume_to_txt(areas, total_volume, txt_filename):
    """
    Save the areas of the polygons and the total volume to a TXT file.
    """
    with open(txt_filename, 'w') as txt_file:
        txt_file.write("Slice Number\tArea (square meters)\n")
        for slice_num, area in areas.items():
            txt_file.write(f"{slice_num}\t{area:.6f}\n")
        txt_file.write(f"\nTotal Volume (cubic meters): {total_volume:.6f}\n")
    print(f"Areas and total volume saved to {txt_filename}")

# Process multiple slices and accumulate areas
slice_nums = input("Enter the slice numbers to process (comma separated): ")
slice_nums = [int(num) for num in slice_nums.split(',')]

total_area = 0
areas = {}

# Iterate through each slice
for slice_num in slice_nums:
    area = process_slice(slice_num, 0.1)  # Pass slice thickness as 0.1 meters
    if area > 0:
        areas[slice_num] = area
        total_area += area

total_volume = total_area * 0.1  # Calculate total volume by multiplying by slice thickness
print(f"Total area of all polygons in physical units: {total_area} square meters")
print(f"Total volume: {total_volume} cubic meters")

# Save all polygons as a triangulated mesh to a PLY file
ply_filename = os.path.join(new_output_dir, 'combined_polygons_triangulated.ply')
save_all_polygons_to_ply(all_polygons, ply_filename)

# Save the areas and total volume to a TXT file
txt_filename = os.path.join(new_output_dir, 'areas_and_total_volume.txt')
save_areas_and_total_volume_to_txt(areas, total_volume, txt_filename)



Enter the slice numbers to process (comma separated): 1
Polygon area in physical units for slice 1: 1.0294111673642088 square meters
Processed image saved at: processed_point_cloud\slice_1_processed.png
Total area of all polygons in physical units: 1.0294111673642088 square meters
Total volume: 0.10294111673642088 cubic meters
All polygons saved as a single PLY file to processed_point_cloud\combined_polygons_triangulated.ply
Areas and total volume saved to processed_point_cloud\areas_and_total_volume.txt


In [3]:
import numpy as np
import open3d as o3d
import os

# Define the output directory where point cloud data and metadata are stored
output_dir = r"C:\Users\wenru\Desktop\point_cloud_info"

# Load point cloud data (points.npy) and metadata (metadata.npy)
points_filename = os.path.join(output_dir, f"{os.path.basename(output_dir)}_points.npy")
metadata_filename = os.path.join(output_dir, f"{os.path.basename(output_dir)}_metadata.npy")

points = np.load(points_filename)
metadata = np.load(metadata_filename, allow_pickle=True)

# Initialize a list to store all points from the slices
all_slices_points = []

# Loop over each slice's metadata
for meta in metadata:
    slice_idx = meta['slice']
    z_value = slice_idx * 0.1  # The thickness of each slice is 0.1 meters
    
    # Extract the points within the current slice's z-range
    slice_points = points[(points[:, 2] >= z_value) & (points[:, 2] < z_value + 0.1)]
    
    # If the slice contains points, adjust the Z value and ensure points are within the slice bounds
    if slice_points.size > 0:
        # Get the bounding box (x_min, y_min, x_max, y_max) from metadata
        x_min, y_min, x_max, y_max = meta['bbox']
        
        # Clip points to be within the bounding box in X and Y directions
        slice_points[:, 0] = np.clip(slice_points[:, 0], x_min, x_max)
        slice_points[:, 1] = np.clip(slice_points[:, 1], y_min, y_max)
        
        # Set the Z value to the actual height of the slice
        slice_points[:, 2] = z_value
        
        # Add the processed points of the slice to the list
        all_slices_points.append(slice_points)

# Combine all points from all slices into a single NumPy array
combined_slices_points = np.vstack(all_slices_points)

# Create a point cloud object from the combined points
combined_point_cloud = o3d.geometry.PointCloud()
combined_point_cloud.points = o3d.utility.Vector3dVector(combined_slices_points)

# Define the output file path for the combined PLY file
output_ply_file = os.path.join(output_dir, 'combined_slices_point_cloud.ply')

# Save the combined point cloud as a PLY file
o3d.io.write_point_cloud(output_ply_file, combined_point_cloud)

print(f"Combined PLY file saved to {output_ply_file}")



Combined PLY file saved to C:\Users\wenru\Desktop\point_cloud_info\combined_slices_point_cloud.ply
